In [2]:
tile_idx = 12099719
varval = 1

In [3]:
import application, getTileVariants
import subprocess

In [4]:
tile = application.Tile(tile_idx)

In [5]:
info = getTileVariants.tile_iteration(tile)#, out="suppress")

Tile 12099719 information:
hg19:chr9:01c4	91424	96791024	15	16
Path: 01c4
Step: 0369
Phase: 0x1

0368	 136139786
0369	 136140011

01c4.00.0369.000+1,60fe6d1a287c91939b151001adfdeada      ttcactgttcggggccactagcaacaaagcttctgtaaacactcatgtgcaggtatttgt
01c4.00.0369.001+1,7214c84c18b994abdd0dbf6a176a7407      ttcactgttcggggccactagcaacaaagcttctgtaaacactcatgtgcaggtatttgt
01c4.00.0369.002+1,34c37138df011524d54aec04d3f2d723      ttcactgttcggggccactagcaacaaagcttctgtaaacactcatgtgcaggtatttgt
01c4.00.0369.003+1,bc42c6d168b007e5278f4308fd4db955      ttcactgttcggggccactagcaacaaagcttctgtaaacactcatgtgcaggtatttgt
01c4.00.0369.004+1,dbac4cca74cdfeecc02209020cf2529b      ttcactgttcggggccactagcaacaaagcttctgtaaacactcatgtgcaggtatttgt
01c4.00.0369.005+2,10bbbac9978c3fa024c501b33bd66c1e      ttcactgttcggggccactagcaacaaagcttctgtaaacactcatgtgcaggtatttgt
                                                         ************************************************************


01c4.00.0369.000+1,60fe6d1a287c91939b15100

In [6]:
info.to_dict()

{'chrom': '9',
 'path': '01c4',
 'position_end': '136140011',
 'position_start': '136139786',
 'step': '0369'}

In [22]:
tile_variant = info.variants.split('\n')[varval]
common_variant = info.variants.split('\n')[0]

In [23]:
print(tile_variant)
print(common_variant)

01c4.00.0369.001+1,7214c84c18b994abdd0dbf6a176a7407,ttcactgttcggggccactagcaacaaagcttctgtaaacactcatgtgcaggtatttgtggacctgtgttttcatttcccttgagtaaataccaaggaatggaaaagatgggtcatatggtaagtgtatgtttaactttgtaagaaactgtgttccaaagtggtgactgaattttccattcccgccaggaatgtacgagaaccacagtccttgccaacacttgatggtcactcttttatc
01c4.00.0369.000+1,60fe6d1a287c91939b151001adfdeada,ttcactgttcggggccactagcaacaaagcttctgtaaacactcatgtgcaggtatttgtggacctgtgttttcatttcccttgagtaaataccaaggaatggaaaagatgggtcatatggtaagtgtatgtttaactttgtaagaaactgccaaactgtgttccaaagtggtgactgaattttccattcccgccaggaatgtacgagaaccacagtccttgccaacacttgatggtcactcttttatc


In [24]:
from Bio import Align
aligner = Align.PairwiseAligner()
aligner.mode = "local"

In [25]:
tile_seq = tile_variant.split(',')[2]
common_seq = common_variant.split(',')[2]

In [26]:
alignments = aligner.align(common_seq, tile_seq)

In [27]:
align = str(alignments[0]).split('\n')

In [28]:
align

['ttcactgttcggggccactagcaacaaagcttctgtaaacactcatgtgcaggtatttgtggacctgtgttttcatttcccttgagtaaataccaaggaatggaaaagatgggtcatatggtaagtgtatgtttaactttgtaagaaactgccaaactgtgttccaaagtggtgactgaattttccattcccgccaggaatgtacgagaaccacagtccttgccaacacttgatggtcactcttttatc',
 '|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||------|||-|-||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||',
 'ttcactgttcggggccactagcaacaaagcttctgtaaacactcatgtgcaggtatttgtggacctgtgttttcatttcccttgagtaaataccaaggaatggaaaagatgggtcatatggtaagtgtatgtttaactttgtaagaaactg------tgt-t-ccaaagtggtgactgaattttccattcccgccaggaatgtacgagaaccacagtccttgccaacacttgatggtcactcttttatc',
 '']

In [16]:
alignment_info = str(alignments[0]).split('\n')[1]

In [17]:
diffs = [i for i, char in enumerate(alignment_info) if char != '|']

In [18]:
indices = [i + int(info.to_dict()['position_start']) - 24 for i in diffs]

In [19]:
print(diffs, indices)

([151, 152, 153, 154, 155, 156, 160, 162, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266], [136139913, 136139914, 136139915, 136139916, 136139917, 136139918, 136139922, 136139924, 136140011, 136140012, 136140013, 136140014, 136140015, 136140016, 136140017, 136140018, 136140019, 136140020, 136140021, 136140022, 136140023, 136140024, 136140025, 136140026, 136140027, 136140028])


In [20]:
tile_dict = info.to_dict()
# chromosome and position and it checks for the particular position
checkstr = "{}:{}-{}".format(tile_dict['chrom'], tile_dict['position_start'], tile_dict['position_end'])
# chromosome:position
# chr:from-to
# 9:136139786-136140011
# GrCH37
outputzip = "/home/kfang/keep/by_id/su92l-4zz18-v96bt8gwbsini81/GRCh37p13.vcf.gz"
#outputzip = "/home/kfang/keep/by_id/su92l-4zz18-v96bt8gwbsini81/GRCh38p7.vcf.gz"

In [21]:
print(checkstr)

9:136139786-136140011


In [218]:
bcfstr = 'bcftools query -f' + '\'%CHROM %ID %POS\\n\'' + ' -r ' + checkstr + ' ' + outputzip

In [219]:
resultquery = subprocess.check_output(bcfstr,shell=True)

In [220]:
rsid_info = [query.split(' ') for query in resultquery.split('\n') if query != ""]

In [221]:
rsid_info

[['9', 'rs138986090', '136139812'],
 ['9', 'rs141515001', '136139833'],
 ['9', 'rs782095868', '136139863'],
 ['9', 'rs782697011', '136139864'],
 ['9', 'rs746655603', '136139873'],
 ['9', 'rs782134971', '136139907'],
 ['9', 'rs149092047', '136139907'],
 ['9', 'rs60937319', '136139907'],
 ['9', 'rs184946026', '136139913'],
 ['9', 'rs187240396', '136139914'],
 ['9', 'rs8176679', '136139955'],
 ['9', 'rs150484570', '136139961'],
 ['9', 'rs587743474', '136139967'],
 ['9', 'rs782144886', '136139968']]

In [222]:
loc_rsid_map = {}
for chrom, rsid, loc in rsid_info:
    try:
        loc_rsid_map[loc].append(rsid)
    except:
        loc_rsid_map[loc] = [rsid]

In [223]:
print(loc_rsid_map)

{'136139907': ['rs782134971', 'rs149092047', 'rs60937319'], '136139913': ['rs184946026'], '136139833': ['rs141515001'], '136139968': ['rs782144886'], '136139967': ['rs587743474'], '136139863': ['rs782095868'], '136139873': ['rs746655603'], '136139914': ['rs187240396'], '136139864': ['rs782697011'], '136139955': ['rs8176679'], '136139961': ['rs150484570'], '136139812': ['rs138986090']}


In [224]:
from operator import itemgetter
from itertools import groupby
def group_continuous_diffs(clustalo_diffs):
        groups = []
        for _, j in groupby(enumerate(clustalo_diffs), lambda item: item[0] - item[1]):
                groups.append(map(itemgetter(1), j))
        return groups

In [225]:
diffs_list = group_continuous_diffs(indices)

In [226]:
loc_rsid_map

{'136139812': ['rs138986090'],
 '136139833': ['rs141515001'],
 '136139863': ['rs782095868'],
 '136139864': ['rs782697011'],
 '136139873': ['rs746655603'],
 '136139907': ['rs782134971', 'rs149092047', 'rs60937319'],
 '136139913': ['rs184946026'],
 '136139914': ['rs187240396'],
 '136139955': ['rs8176679'],
 '136139961': ['rs150484570'],
 '136139967': ['rs587743474'],
 '136139968': ['rs782144886']}

In [244]:
for diff in diffs_list:
    if len(diff) > 1:
        rsid_search_range = (diff[0] - 10, diff[-1] + 10)
        print_fmt ="{}-{}, search range: {}-{}".format(diff[0], diff[-1], rsid_search_range[0], rsid_search_range[1])
    else:
        rsid_search_range = (diff[0] - 10, diff[0] + 10)
        print_fmt = "{}, search range: {}-{}".format(diff[0], rsid_search_range[0], rsid_search_range[1])
        
    possible_rsids = []
    for location in range(rsid_search_range[0], rsid_search_range[1] + 1):
        try:
            possible_rsids.append((location, loc_rsid_map[str(location)]))
        except KeyError:
            pass
        
    print(print_fmt)
    for pos in possible_rsids:
        print(pos)
    print

136139913-136139918, search range: 136139903-136139928
(136139907, ['rs782134971', 'rs149092047', 'rs60937319'])
(136139913, ['rs184946026'])
(136139914, ['rs187240396'])

136139922, search range: 136139912-136139932
(136139913, ['rs184946026'])
(136139914, ['rs187240396'])

136139924, search range: 136139914-136139934
(136139914, ['rs187240396'])

